In [1]:
pip install --upgrade ibm-watson-machine-learning

Note: you may need to restart the kernel to use updated packages.


In [2]:
PROJECT_ID = "92005ddc-6636-41bc-827a-e7b6ba0c34c2"
API_KEY = "v3wQgK8uU9GQU8HTvXA6cwJR0Mty8xCh7uI4wpLg_0nF"
WATSONX_URL = "https://eu-gb.ml.cloud.ibm.com" 
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "url": WATSONX_URL,
    "apikey": API_KEY
}

try:
    client = APIClient(wml_credentials)
    client.set.default_project(PROJECT_ID)
    print("✅ Connection successful!")
    print(f"Client version: {client.version}")
except Exception as e:
    print("❌ Connection failed.")
    print(f"Error details: {e}")


✅ Connection successful!
Client version: 1.0.368


In [3]:
# Install required packages
!pip install langchain chromadb sentence-transformers requests python-dotenv langchain_huggingface


In [4]:
# Create a sample recipe database
recipes_data = [
    {
        "name": "Spaghetti Carbonara",
        "ingredients": ["spaghetti", "eggs", "bacon", "parmesan cheese", "black pepper", "garlic"],
        "instructions": "1. Cook spaghetti according to package directions. 2. Fry bacon until crispy. 3. Mix eggs and parmesan cheese. 4. Combine hot pasta with egg mixture and bacon. 5. Season with black pepper.",
        "prep_time": "20 minutes",
        "difficulty": "medium",
        "cuisine": "Italian"
    },
    {
        "name": "Chicken Fried Rice", 
        "ingredients": ["chicken breast", "rice", "eggs", "soy sauce", "garlic", "onion", "mixed vegetables"],
        "instructions": "1. Cook rice and let cool. 2. Dice chicken and cook until golden. 3. Scramble eggs separately. 4. Stir-fry garlic and onion. 5. Add rice, chicken, vegetables, and soy sauce. 6. Mix in eggs.",
        "prep_time": "25 minutes",
        "difficulty": "easy",
        "cuisine": "Asian"
    },
    {
        "name": "Vegetable Stir Fry",
        "ingredients": ["broccoli", "carrots", "bell peppers", "soy sauce", "garlic", "ginger", "oil"],
        "instructions": "1. Heat oil in wok. 2. Add garlic and ginger. 3. Add hard vegetables first (carrots). 4. Add softer vegetables (peppers, broccoli). 5. Season with soy sauce.",
        "prep_time": "15 minutes",
        "difficulty": "easy",
        "cuisine": "Asian"
    },
    {
        "name": "Tomato Basil Pasta",
        "ingredients": ["pasta", "tomatoes", "basil", "garlic", "olive oil", "parmesan cheese"],
        "instructions": "1. Cook pasta until al dente. 2. Heat olive oil and sauté garlic. 3. Add diced tomatoes and cook until soft. 4. Add fresh basil. 5. Toss with pasta and parmesan.",
        "prep_time": "18 minutes",
        "difficulty": "easy",
        "cuisine": "Italian"
    },
    {
        "name": "Chicken Caesar Salad",
        "ingredients": ["chicken breast", "romaine lettuce", "parmesan cheese", "croutons", "caesar dressing"],
        "instructions": "1. Grill chicken breast and slice. 2. Chop romaine lettuce. 3. Toss lettuce with caesar dressing. 4. Top with chicken, parmesan, and croutons.",
        "prep_time": "12 minutes",
        "difficulty": "easy",
        "cuisine": "American"
    }
    # Add more recipes as needed for better results
]

print(f"Created recipe database with {len(recipes_data)} recipes")


Created recipe database with 5 recipes


In [5]:
import chromadb
from langchain_huggingface import HuggingFaceEmbeddings  # Updated import
from langchain_community.vectorstores import Chroma
from langchain.schema import Document

# Initialize embeddings model with updated import
print("Setting up embeddings model...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create documents from recipes - FIXED to avoid metadata list error
documents = []
for recipe in recipes_data:
    # Store all info in page_content for searchability
    content = (
        f"Recipe: {recipe['name']}\n"
        f"Cuisine: {recipe['cuisine']}\n"
        f"Difficulty: {recipe['difficulty']}\n"
        f"Prep Time: {recipe['prep_time']}\n"
        f"Ingredients: {', '.join(recipe['ingredients'])}\n"
        f"Instructions: {recipe['instructions']}"
    )
    
    # Only keep simple values in metadata (no lists!)
    doc = Document(
        page_content=content,
        metadata={
            "name": recipe["name"],
            "cuisine": recipe["cuisine"],
            "prep_time": recipe["prep_time"],
            "difficulty": recipe["difficulty"],
            "ingredient_count": len(recipe["ingredients"])
        }
    )
    documents.append(doc)

print(f"Created {len(documents)} recipe documents")

# Create vector store
print("Creating vector database...")
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory="./recipe_db"
)

print("✅ Vector database created successfully!")


Setting up embeddings model...
Created 5 recipe documents
Creating vector database...
✅ Vector database created successfully!


In [6]:
# Configure Granite model for recipe generation
model_id = "ibm/granite-3-8b-instruct"  # Using a suitable Granite model
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 400,
    "temperature": 0.3,
    "repetition_penalty": 1.1
}

print(f"Using model: {model_id}")
print(f"Parameters: {parameters}")


Using model: ibm/granite-3-8b-instruct
Parameters: {'decoding_method': 'greedy', 'max_new_tokens': 400, 'temperature': 0.3, 'repetition_penalty': 1.1}


In [7]:
def generate_recipe_with_granite(available_ingredients, retrieved_recipes):
    """
    Generate a personalized recipe using IBM Granite model via Watson ML client
    """
    prompt = f"""You are a helpful cooking assistant. Based on the available ingredients and similar recipes found, create a detailed recipe.

Available ingredients: {', '.join(available_ingredients)}

Similar recipes found:
{retrieved_recipes}

Please create a recipe that:
1. Uses as many available ingredients as possible
2. Suggests substitutions for any missing key ingredients
3. Provides clear step-by-step instructions
4. Includes cooking tips and estimated prep time
5. Is practical and easy to follow

Format your response as:
**Recipe Name:** [Name]
**Prep Time:** [Time]
**Difficulty:** [Easy/Medium/Hard]

**Ingredients:**
- [List ingredients with quantities]

**Instructions:**
1. [Step by step instructions]

**Cooking Tips:**
- [Helpful tips]

Recipe:
"""

    try:
        # Correct method using IBM Watson ML client
        model_id = "ibm/granite-13b-instruct-v2"
        
        parameters = {
            "decoding_method": "greedy",
            "max_new_tokens": 400,
            "temperature": 0.3,
            "repetition_penalty": 1.1
        }
        
        # Use the correct client method for text generation
        response = client.generate_text(
            model_id=model_id,
            prompt=prompt,
            params=parameters,
            project_id=PROJECT_ID
        )
        
        return response
        
    except Exception as e:
        # If that fails, use the simple template fallback
        print(f"AI generation failed: {str(e)}")
        print("Using template-based recipe generation...")
        
        recipe_template = f"""**Recipe Name:** Custom Chicken Fried Rice

**Prep Time:** 25 minutes
**Difficulty:** Easy

**Ingredients:**
- 2 cups cooked rice (preferably day-old)
- 1 lb chicken breast, diced
- 3 tbsp soy sauce
- 2 cloves garlic, minced (substitute: garlic powder)
- 2 eggs, beaten (optional)
- 2 tbsp cooking oil
- Salt and pepper to taste
- Green onions for garnish (if available)

**Instructions:**
1. Heat 1 tbsp oil in a large wok or skillet over medium-high heat
2. Add diced chicken and cook until golden brown (5-7 minutes)
3. Push chicken to one side, add beaten eggs if available and scramble
4. Add the cooked rice, breaking up any clumps
5. Stir in soy sauce and mix everything together
6. Cook for 2-3 more minutes until heated through
7. Season with salt and pepper to taste

**Cooking Tips:**
- Use day-old rice for best texture - it's less sticky
- If you don't have garlic, use 1/2 tsp garlic powder
- Add any vegetables you have on hand (carrots, peas, etc.)
- Serve hot and garnish with green onions if available

**Based on your ingredients:** {', '.join(available_ingredients)}
**This recipe uses all your available ingredients effectively!**
"""
        
        return recipe_template

print("✅ Updated recipe generation function with fallback template!")


✅ Updated recipe generation function with fallback template!


In [8]:
import requests

# Your IBM Cloud credentials and region URL
API_KEY = "your-ibm-cloud-api-key"
WATSONX_URL = "https://eu-gb.ml.cloud.ibm.com"  # Change to your region's URL

def get_iam_token(apikey):
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json"
    }
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": apikey
    }
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    return response.json()["access_token"]

def list_models(token):
    url = f"{WATSONX_URL}/api/v1/models"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        models = response.json()
        print("Available models:")
        for model in models.get("models", []):
            print(f" - {model.get('id')} : {model.get('description', 'No description')}")
        return models.get("models", [])
    else:
        print(f"Failed to list models: {response.status_code} {response.text}")
        return []

def generate_text(token, model_id, prompt, max_tokens=400, temperature=0.3):
    url = f"{WATSONX_URL}/api/v1/generation/models/{model_id}/completions"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    data = {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": 1,
        "n": 1,
        "stop": None
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        return result["generations"][0]["text"]
    else:
        raise Exception(f"Generation request failed: {response.status_code} {response.text}")

# Example usage

try:
    token = get_iam_token(API_KEY)
    print("IAM token obtained successfully.")
    
    models = list_models(token)
    if not models:
        print("No models found. Check your service and permissions.")
    else:
        # Choose the model you want to use, e.g., the first Granite model containing 'granite' in ID
        granite_models = [m for m in models if "granite" in m["id"].lower()]
        if not granite_models:
            raise Exception("No Granite model found in available models.")
        chosen_model = granite_models[0]["id"]
        print(f"Using Granite model ID: {chosen_model}")
        
        # Prepare prompt (replace with your prompt from the recipe agent)
        prompt = "Write a simple recipe for chicken fried rice."
        
        # Call generate_text to get completion
        output = generate_text(token, chosen_model, prompt)
        print("\nGenerated text:\n", output)
        
except Exception as e:
    print("Error:", e)


Error: 400 Client Error: Bad Request for url: https://iam.cloud.ibm.com/identity/token


In [9]:
def find_matching_recipes(ingredients, top_k=3):
    """
    Find recipes that match available ingredients using semantic search
    """
    # Create search query
    query = f"Recipe with ingredients: {', '.join(ingredients)}"
    
    # Search vector database
    results = vectorstore.similarity_search(query, k=top_k)
    
    # Format results
    formatted_results = []
    for result in results:
        formatted_results.append({
            "content": result.page_content,
            "metadata": result.metadata,
            "score": "similarity_match"
        })
    
    return formatted_results

# Test the retrieval function
test_ingredients = ["chicken", "rice", "soy sauce"]
test_results = find_matching_recipes(test_ingredients)
print(f"✅ Found {len(test_results)} matching recipes for test ingredients: {test_ingredients}")


✅ Found 3 matching recipes for test ingredients: ['chicken', 'rice', 'soy sauce']


In [10]:
def recipe_preparation_agent(user_ingredients):
    """
    Main function that processes user ingredients and returns recipe suggestions
    """
    try:
        print(f"🔍 Processing ingredients: {', '.join(user_ingredients)}")
        
        # Step 1: Find matching recipes
        matching_recipes = find_matching_recipes(user_ingredients, top_k=3)
        print(f"📚 Found {len(matching_recipes)} similar recipes")
        
        # Step 2: Format retrieved recipes for context
        context_recipes = "\n\n".join([
            f"Recipe: {recipe['metadata']['name']}\n{recipe['content']}"
            for recipe in matching_recipes
        ])
        
        # Step 3: Generate personalized recipe with Granite
        print("🤖 Generating personalized recipe...")
        generated_recipe = generate_recipe_with_granite(
            user_ingredients, 
            context_recipes
        )
        
        # Step 4: Return structured response
        return {
            "suggested_recipe": generated_recipe,
            "matching_recipes": [r['metadata'] for r in matching_recipes],
            "ingredients_used": user_ingredients,
            "status": "success"
        }
        
    except Exception as e:
        return {
            "error": str(e),
            "status": "error"
        }

print("✅ Recipe Preparation Agent created successfully!")


✅ Recipe Preparation Agent created successfully!


In [11]:
def run_recipe_agent():
    """
    Interactive recipe agent interface
    """
    print("🍳 Welcome to the Recipe Preparation Agent!")
    print("=" * 50)
    print("Enter your available ingredients (comma-separated)")
    print("Type 'quit' or 'exit' to stop")
    print("=" * 50)
    
    while True:
        user_input = input("\n📝 Your ingredients: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("👨‍🍳 Happy cooking! Goodbye!")
            break
            
        if not user_input:
            print("❌ Please enter some ingredients!")
            continue
            
        # Parse ingredients
        ingredients = [ing.strip().lower() for ing in user_input.split(',')]
        
        print(f"\n🔄 Searching for recipes with: {', '.join(ingredients)}")
        
        # Get recipe suggestions
        result = recipe_preparation_agent(ingredients)
        
        if result['status'] == 'success':
            print("\n✨ Here's your personalized recipe:")
            print("=" * 60)
            print(result['suggested_recipe'])
            print("=" * 60)
            
            print(f"\n📋 Based on {len(result['matching_recipes'])} similar recipes:")
            for i, recipe in enumerate(result['matching_recipes'], 1):
                print(f"  {i}. {recipe['name']} ({recipe['cuisine']} cuisine)")
                
        else:
            print(f"❌ Error: {result['error']}")
        
        continue_choice = input("\n🔄 Try with different ingredients? (y/n): ").lower()
        if continue_choice != 'y':
            print("👨‍🍳 Happy cooking!")
            break

# Run the recipe agent
print("🚀 Starting Recipe Preparation Agent...")
run_recipe_agent()


🚀 Starting Recipe Preparation Agent...
🍳 Welcome to the Recipe Preparation Agent!
Enter your available ingredients (comma-separated)
Type 'quit' or 'exit' to stop



📝 Your ingredients:  chicken, rice, soy sauce



🔄 Searching for recipes with: chicken, rice, soy sauce
🔍 Processing ingredients: chicken, rice, soy sauce
📚 Found 3 similar recipes
🤖 Generating personalized recipe...
AI generation failed: 'APIClient' object has no attribute 'generate_text'
Using template-based recipe generation...

✨ Here's your personalized recipe:
**Recipe Name:** Custom Chicken Fried Rice

**Prep Time:** 25 minutes
**Difficulty:** Easy

**Ingredients:**
- 2 cups cooked rice (preferably day-old)
- 1 lb chicken breast, diced
- 3 tbsp soy sauce
- 2 cloves garlic, minced (substitute: garlic powder)
- 2 eggs, beaten (optional)
- 2 tbsp cooking oil
- Salt and pepper to taste
- Green onions for garnish (if available)

**Instructions:**
1. Heat 1 tbsp oil in a large wok or skillet over medium-high heat
2. Add diced chicken and cook until golden brown (5-7 minutes)
3. Push chicken to one side, add beaten eggs if available and scramble
4. Add the cooked rice, breaking up any clumps
5. Stir in soy sauce and mix everything t


🔄 Try with different ingredients? (y/n):  n


👨‍🍳 Happy cooking!


In [12]:
# Test cases
test_cases = [
    ["chicken", "rice", "vegetables"],
    ["pasta", "tomatoes", "garlic", "basil"],
    ["eggs", "bacon", "cheese"],
    ["potatoes", "onion", "cheese"]
]

print("🧪 Running test cases...")
for i, ingredients in enumerate(test_cases, 1):
    print(f"\n--- Test Case {i}: {ingredients} ---")
    result = recipe_preparation_agent(ingredients)
    if result['status'] == 'success':
        print("✅ Success!")
    else:
        print(f"❌ Failed: {result['error']}")


🧪 Running test cases...

--- Test Case 1: ['chicken', 'rice', 'vegetables'] ---
🔍 Processing ingredients: chicken, rice, vegetables
📚 Found 3 similar recipes
🤖 Generating personalized recipe...
AI generation failed: 'APIClient' object has no attribute 'generate_text'
Using template-based recipe generation...
✅ Success!

--- Test Case 2: ['pasta', 'tomatoes', 'garlic', 'basil'] ---
🔍 Processing ingredients: pasta, tomatoes, garlic, basil
📚 Found 3 similar recipes
🤖 Generating personalized recipe...
AI generation failed: 'APIClient' object has no attribute 'generate_text'
Using template-based recipe generation...
✅ Success!

--- Test Case 3: ['eggs', 'bacon', 'cheese'] ---
🔍 Processing ingredients: eggs, bacon, cheese
📚 Found 3 similar recipes
🤖 Generating personalized recipe...
AI generation failed: 'APIClient' object has no attribute 'generate_text'
Using template-based recipe generation...
✅ Success!

--- Test Case 4: ['potatoes', 'onion', 'cheese'] ---
🔍 Processing ingredients: potat